# Get the data 

In [137]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [146]:
def get_data(**kwargs):
    ''' Retrieve XML data using King County's API
    
        Kwargs
        ======
        Business_Name - string
        Business_Address - string
        Zip_Code - string
        Inspection_Start - string with valid date in format MM/DD/YYYY
        Inspection_End - string with valid date in format MM/DD/YYYY
        Violation_Points - string with Integer <= 999
        Violation_Red_Points - string with Integer <=999
        City - string
        
        Example
        =======
        Returns inspection data in zip code 98101 since 1/1/2016
        >>> url_formatter(Zip_Code='98101', Inspection_Start=1/1/2016)
        
        Notes
        =====
        The API appears to break using parameter Inspection_Closed_Business
    '''
    if kwargs == None:
        print('Oh no! You forgot to select at least one parameter')
        return
        
    else:
        url = 'http://info.kingcounty.gov/health/ehs/foodsafety/inspections/XmlRest.aspx?'
        for key in kwargs:
            s = '='.join([key, kwargs[key]])
            url = '&'.join([url, s])
    # retrieve data
    response = requests.get(url)
    root = ET.fromstring(response.content)
    # Remove the King County data disclaimer
    element = root.find('Disclaimer')
    root.remove(element)
    return root

In [141]:
root = get_data(Zip_Code='98101', Inspection_Start='1/1/2016', Inspection_End='5/1/2016')

In [142]:
def get_element_text(attribute):
    if business.find(attribute) is not None:
        return business.find(attribute).text
    else:
        return None

In [143]:
name = []
address = []
city = []
zip_code = []
longitude = []
latitude = []
    
# get all the things    
for i in range(0, len(root)):
    business = root.getchildren()[i]
    name.append(get_element_text('Name'))
    address.append(get_element_text('Address'))
    city.append(get_element_text('City'))
    zip_code.append(get_element_text('Zip_Code'))
    longitude.append(get_element_text('Longitude'))
    latitude.append(get_element_text('Latitude'))

In [144]:
business_dict = {'Name': name, 'Address': address, 'City': city, 'Zip_Code': zip_code,
                'Longitude': longitude, 'Latitude': latitude}
df = pd.DataFrame(business_dict)

In [145]:
df.head()

,Address,City,Latitude,Longitude,Name,Zip_Code
0,1908 3RD AVE,Seattle,47.6120226711,-122.3399331044,3RD AVE CAFE,98101
1,1524 3RD AVE,SEATTLE,47.6105993100,-122.3380513100,7-ELEVEN #36683A,98101
2,1830 MINOR AVE,Seattle,47.6172086272,-122.3308116303,ALFI'S FOOD & DELI,98101
3,1517 PIKE PL,SEATTLE,47.6086943588,-122.3408823957,ATHENIAN INN,98101
4,1420 5TH AVE STE 1700,SEATTLE,None,None,AVANTI MARKETS NW @ GBRE,98101
